In [ ]:
import os

os.environ["WIKIDATA_CACHE_FILE"] = "../.wikidata-cache.json"

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("bp-stats-review-2021-all-data.xlsx")

In [ ]:
df = shifter_ds.datasource()
df

In [ ]:
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

# Build App
app = JupyterDash("BP statistical review")
app.layout = html.Div(
    [
        html.H1("BP statistical review"),
        html.Label(
            [
                "Region",
                dcc.Dropdown(
                    id="Region-dropdown",
                    clearable=False,
                    value="plasma",
                    options=[{"label": c, "value": c} for c in set(df["Region"])],
                ),
            ]
        ),
        html.Label(
            [
                "Indicator",
                dcc.Dropdown(
                    id="Indicator-dropdown",
                    clearable=False,
                    value="plasma",
                    options=[{"label": c, "value": c} for c in set(df["Type"])],
                ),
            ]
        ),
        dcc.Graph(id="graph"),
    ]
)

# Define callback to update graph
@app.callback(
    Output("graph", "figure"),
    [Input("Region-dropdown", "value"), Input("Indicator-dropdown", "value")],
)
def update_figure(region, indicator):
    fig = px.line(
        df.query(f'Region == "{region}" and Type == "{indicator}"'), x="Year", y="Value"
    )
    fig.update_layout(
        yaxis_type="log",
        xaxis={
            "rangeslider": {"visible": True},
        },
    )
    return fig


# Run app and display result inline in the notebook
app.run_server(mode="inline", port=8070)